## Fixed Data:

In [1]:
pip install fredapi

In [117]:
# Progress Bar
pip install tqdm

In [2]:

import json
import requests
import pandas as pd
from fredapi import Fred

API_KEY = '1vGb0FOsg2hEIGrEhpueLCcWd1Y'
FRED_API_KEY = 'b13a905b98b5dfc1f3910d6a0d875ae3'

rootUrl = 'https://api.glassnode.com'

symbolList = ['BTC', 'ETH', 'LTC', 'AAVE', 'ABT', 'AMPL', 'ANT', 'ARMOR', 'BADGER', 'BAL', 'BAND', 'BAT', 'BIX', 'BNT', 'BOND', 'BRD', 'BUSD', 'BZRX', 'CELR', 'CHSB', 'CND', 'COMP', 'CREAM', 'CRO', 'CRV', 'CVC', 'CVP', 'DAI', 'DDX', 'DENT', 'DGX', 'DHT', 'DMG', 'DODO', 'DOUGH', 'DRGN', 'ELF', 'ENG', 'ENJ', 'EURS', 'FET', 'FTT', 'FUN', 'GNO', 'GUSD', 'HEGIC', 'HOT', 'HPT', 'HT', 'HUSD', 'INDEX', 'KCS', 'LAMB', 'LBA', 'LDO', 'LEO', 'LINK', 'LOOM', 'LRC', 'MANA', 'MATIC',
              'MCB', 'MCO', 'MFT', 'MIR', 'MKR', 'MLN', 'MTA', 'MTL', 'MX', 'NDX', 'NEXO', 'NFTX', 'NMR', 'Nsure', 'OCEAN', 'OKB', 'OMG', 'PAY', 'PERP', 'PICKLE', 'PNK', 'PNT', 'POLY', 'POWR', 'PPT', 'QASH', 'QKC', 'QNT', 'RDN', 'REN', 'REP', 'RLC', 'ROOK', 'RPL', 'RSR', 'SAI', 'SAN', 'SNT', 'SNX', 'STAKE', 'STORJ', 'sUSD', 'SUSHI', 'TEL', 'TOP', 'UBT', 'UMA', 'UNI', 'USDC', 'USDK', 'USDP', 'USDT', 'UTK', 'VERI', 'WaBi', 'WAX', 'WBTC', 'WETH', 'wNXM', 'WTC', 'YAM', 'YFI', 'ZRX']

symbolListShort = ['BTC', 'ETH', 'LTC']

endPoints = {
    'Price': '/v1/metrics/market/price_usd_close',
    'MarketCap': '/v1/metrics/market/marketcap_usd',
    'TransactionCount': '/v1/metrics/transactions/count',
    'ActiveAddress': '/v1/metrics/addresses/active_count',
}

### <font color='red'> It seems the risk-free rate is daily rate. is it in percentage or in unit 1? need to adjust to weekly rate and select Monday

In [3]:
fred = Fred(api_key=FRED_API_KEY)
Risk_free_rate = fred.get_series('DGS1MO').to_frame().reset_index().rename(columns={'index':'Date', 
                                                                                    0:'RiskFree'})
Risk_free_rate['RiskFree']= Risk_free_rate['RiskFree'] / 100

## Get Multiple Attributes for Multiple Coins:

In [104]:
### Define a Method to Pull Data According to Input Prameter(coin, feature)

def PullData(coin, feature):
    print(f"Pulling {coin}'s {feature}")

    # Assemble full request url and get responses:
    reqUrl = rootUrl + endPoints[feature]
    res = requests.get(reqUrl,
                       params={'a': coin, 
                               'api_key': API_KEY, 
                               'i':'1w'})
    data_raw = pd.read_json(res.text, 
                            convert_dates=['t'])

    # Rename column "v" according to features
    data = data_raw.rename(columns={"v" : f"{feature}", 
                                    't':'Date'})

    # Return a dataframe
    return data

In [118]:
# Effect:
PullData('BTC', 'Price').head()

Pulling BTC's Price


,Date,Price
0,2010-07-12,0.08584
1,2010-07-19,0.05050
2,2010-07-26,0.06110
3,2010-08-02,0.06090
4,2010-08-09,0.06529


In [124]:
### Define a Method to Get Features(Listed in "endPoints"(Constant)) of Coins(Listed in "coinList"(Method Prameter)) 

def PullStart(coinList):
    # Initialization
    from functools import reduce
    from tqdm import tqdm

    colName = ["Date"]
    for key, value in endPoints.items():
        colName.append(key) 
    fullFrame = pd.DataFrame(columns = colName.append("Asset"))

    # Data Pull
    for i in tqdm(coinList):
        print(f"Start Pulling {i}'s data:")

        # Preparation for merging
        frameName = f"{i}_Frames"
        locals()[frameName] = []

        # Get data
        for j in list(endPoints.keys()):
            locals()[frameName].append(PullData(i, 
                                                j))

        # Merge 2-column dataframes to a single dataframe
        dfName = f"{i}_All"
        # Outer Merge Glassnode to Ensure Not Missing Any Values.
        # Left Merge Fred to ensure all the records in api stored.
        locals()[dfName] = reduce(lambda left, right:
                                  pd.merge(left,right, 
                                           on='Date', 
                                           how='outer'), 
                                  locals()[frameName]).merge(Risk_free_rate, 
                                                             how = 'left').sort_values(by=['Date'])
        locals()[dfName]['Asset'] = i

        # Merge All Coins into a single Dataframe
        fullFrame = fullFrame.append(locals()[dfName],
                                     ignore_index=True)

        print(f"{i} data successfully pulled\n")
    print('All Frames Merged')
    print('\nALL DATA SUCCESSFULLY PULLED!')

    return fullFrame

In [125]:
# Effect:
df = PullStart(symbolListShort)
df

  0%|          | 0/3 [00:00<?, ?it/s]

Start Pulling BTC's data:
Pulling BTC's Price
Pulling BTC's MarketCap
Pulling BTC's TransactionCount
Pulling BTC's ActiveAddress


 33%|███▎      | 1/3 [00:02<00:05,  2.78s/it]

BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price
Pulling ETH's MarketCap
Pulling ETH's TransactionCount
Pulling ETH's ActiveAddress


 67%|██████▋   | 2/3 [00:04<00:01,  1.99s/it]

ETH data successfully pulled

Start Pulling LTC's data:
Pulling LTC's Price
Pulling LTC's MarketCap
Pulling LTC's TransactionCount
Pulling LTC's ActiveAddress


100%|██████████| 3/3 [00:06<00:00,  2.17s/it]

LTC data successfully pulled

All Frames Merged

ALL DATA SUCCESSFULLY PULLED!


,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Asset
0,2008-12-29,NaN,NaN,NaN,0.0,0.0004,BTC
1,2009-01-05,NaN,NaN,NaN,0.0,0.0005,BTC
2,2009-01-12,NaN,NaN,19.0,34.0,0.0004,BTC
3,2009-01-19,NaN,NaN,9.0,63.0,NaN,BTC
4,2009-01-26,NaN,NaN,5.0,61.0,0.0002,BTC
...,...,...,...,...,...,...,...
1595,2022-03-28,128.926292,9.023978e+09,807733.0,1566202.0,0.0019,LTC
1596,2022-04-04,111.343007,7.798857e+09,841901.0,1576883.0,0.0018,LTC
1597,2022-04-11,109.018201,7.641529e+09,832321.0,1652475.0,0.0022,LTC
1598,2022-04-18,104.648982,7.340578e+09,830602.0,1562294.0,0.0038,LTC


## <font color='red'> Imputation for NA values -- to be resolved

## Calculate Returns

In [8]:
### Define a Method to Add Return Column

def AddReturn(sample):
    fullFrame = pd.DataFrame(columns = list(sample.columns).append('Return'))
    for index in sample.groupby('Asset').count().index.values.tolist():
        syntax = f"Asset == '{index}'"
        sagment = sample.query(syntax)
        # Only Select 53 Weeks for Test
        temp = sagment[-53:-1].reset_index().drop('index', axis=1)
        sagment = sagment[-52:].reset_index().drop('index', axis=1)
        sagment['Return'] = (sagment['Price'] - temp['Price'])/temp['Price']
        fullFrame = fullFrame.append(sagment,
                                     ignore_index=True)
    return fullFrame

In [9]:
# Effect:

AddReturn(df)

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Asset,Return
0,2021-05-03,58218.566077,1.088991e+12,2038316.0,6622607.0,0.0002,BTC,0.028357
1,2021-05-10,46406.331544,8.683518e+11,1908851.0,6062366.0,0.0002,BTC,-0.202895
2,2021-05-17,34888.678313,6.530094e+11,1607054.0,4960283.0,0.0000,BTC,-0.248191
3,2021-05-24,35639.981455,6.672647e+11,1508758.0,4981835.0,0.0001,BTC,0.021534
4,2021-05-31,35797.422390,6.704378e+11,1582789.0,4893673.0,NaN,BTC,0.004418
...,...,...,...,...,...,...,...,...
151,2022-03-28,128.926292,9.023978e+09,807733.0,1566202.0,0.0019,LTC,-0.002438
152,2022-04-04,111.343007,7.798857e+09,841901.0,1576883.0,0.0018,LTC,-0.136382
153,2022-04-11,109.018201,7.641529e+09,832321.0,1652475.0,0.0022,LTC,-0.020880
154,2022-04-18,104.648982,7.340578e+09,830602.0,1562294.0,0.0038,LTC,-0.040078


## MOM Factor

### Get LargeCap Sample(maket cap > 10000)

In [95]:
### Define a Method to Get LargeCoin List

def getLargeSample():
    from tqdm import tqdm
    largeList = []
    print('Finding All Coins with More Than 1m Cap......')
    for i in tqdm(symbolList):
        if pd.read_json(requests.get(rootUrl + endPoints['MarketCap'], 
                                    params={'a': i, 
                                            'api_key': API_KEY, 
                                            'i':'1w'}).text, 
                        convert_dates=['t']).iloc[-1,1] > 1000000:
            largeList.append(i)
    print("\nDone!")

    return largeList

largeList = getLargeSample()

Finding All Coins with More Than 1m Cap......


100%|██████████| 124/124 [00:50<00:00,  2.47it/s]


Done!


In [106]:
# Get largeCapSample_raw Data(Without Return)
largeCapSample_raw = PullStart(largeList)

  0%|          | 0/120 [00:00<?, ?it/s]

Start Pulling BTC's data:
Pulling BTC's Price
Pulling BTC's MarketCap
Pulling BTC's TransactionCount
Pulling BTC's ActiveAddress


  1%|          | 1/120 [00:03<06:20,  3.20s/it]

BTC data successfully pulled

Start Pulling ETH's data:
Pulling ETH's Price
Pulling ETH's MarketCap
Pulling ETH's TransactionCount
Pulling ETH's ActiveAddress


  2%|▏         | 2/120 [00:04<04:12,  2.14s/it]

...

YAM data successfully pulled

Start Pulling YFI's data:
Pulling YFI's Price
Pulling YFI's MarketCap
Pulling YFI's TransactionCount
Pulling YFI's ActiveAddress


 99%|█████████▉| 119/120 [02:23<00:01,  1.05s/it]

YFI data successfully pulled

Start Pulling ZRX's data:
Pulling ZRX's Price
Pulling ZRX's MarketCap
Pulling ZRX's TransactionCount
Pulling ZRX's ActiveAddress


100%|██████████| 120/120 [02:24<00:00,  1.21s/it]

ZRX data successfully pulled

All Frames Merged

ALL DATA SUCCESSFULLY PULLED!


In [25]:
# Add Return to largeCapSample_raw
largeCapSample = AddReturn(largeCapSample_raw)
largeCapSample

,Date,Price,MarketCap,TransactionCount,ActiveAddress,RiskFree,Asset,Return
0,2021-05-03,459.310213,5.735887e+09,0.0,9738.0,0.0002,AAVE,-0.049263
1,2021-05-10,525.296787,6.648981e+09,0.0,10036.0,0.0002,AAVE,0.143665
2,2021-05-17,296.307038,3.779533e+09,0.0,12152.0,0.0000,AAVE,-0.435925
3,2021-05-24,324.282384,4.141262e+09,0.0,11055.0,0.0001,AAVE,0.094413
4,2021-05-31,365.167460,4.669538e+09,0.0,7849.0,NaN,AAVE,0.126079
...,...,...,...,...,...,...,...,...
6235,2022-03-28,34.426101,5.773133e+07,0.0,90.0,0.0019,wNXM,0.077775
6236,2022-04-04,33.251072,5.576085e+07,0.0,195.0,0.0018,wNXM,-0.034132
6237,2022-04-11,27.854533,4.671105e+07,0.0,255.0,0.0022,wNXM,-0.162297
6238,2022-04-18,28.499363,4.779241e+07,0.0,191.0,0.0038,wNXM,0.023150


### Reshape Dataframe

In [138]:
# Get Return DataFrame
returnAll = largeCapSample.loc[:,['Date', 'Return', 'Asset']].pivot(index="Date", columns=["Asset"], values='Return').reset_index().rename_axis(None, axis=1).sort_values('Date')
returnAll.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-03,-0.049263,0.022970,0.112456,0.069679,0.080898,-0.024938,0.079109,0.001570,0.126182,...,0.406858,NaN,0.032826,0.141704,0.070760,-0.145964,0.102180,0.046813,0.010270,0.348608
2,2021-05-10,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
3,2021-05-17,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
4,2021-05-24,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871


In [139]:
priceAll = largeCapSample.loc[:,['Date', 'Price', 'Asset']].pivot(index="Date", columns=["Asset"], values='Price').reset_index().rename_axis(None, axis=1).sort_values('Date')
priceAll.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-04-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-03,459.310213,0.228918,1.190291,10.185312,1.136358,35.192097,66.211436,17.652240,1.414453,...,24.769653,NaN,58296.148718,1.963506,0.599371,1.980944,53279.650539,1.955483,1.026809,117.702644
2,2021-05-10,525.296787,0.189665,0.854800,7.695547,0.768935,26.170739,54.122365,14.034590,1.139028,...,26.309891,NaN,46370.827072,1.641239,0.408041,2.109437,65759.108989,1.572884,1.019278,99.996818
3,2021-05-17,296.307038,0.098163,0.549472,3.380071,0.417567,10.523989,25.810436,6.810641,0.564851,...,14.539290,NaN,34975.530687,0.667486,0.182926,1.302382,32922.121717,0.807320,0.999703,58.923315
4,2021-05-24,324.282384,0.135402,0.804883,4.151059,0.460737,11.759670,27.612201,7.390893,0.703204,...,17.283173,NaN,35641.705527,0.804181,0.221759,0.938102,41882.102419,0.937811,1.005210,73.175135


### Calculate Momentum Matrix(Cumulative 2 week Return)

In [140]:
### Define a Method to Transform Price Matrix to MOM Matrix

def getMomMatrix(priceAll):
    # Initialization
    from tqdm import tqdm
    momMatrix = priceAll.copy()
    
    # Iterate Through priceAll 
    for i in tqdm(range(0, priceAll.shape[0]-2)):
        for j in range(1, priceAll.shape[1]):
            momMatrix.iloc[i+2, j] = (priceAll.iloc[i+2, j] - priceAll.iloc[i,j]) / priceAll.iloc[i,j]
    return momMatrix.drop(momMatrix.head(2).index).reset_index().drop('index',
                                                                      axis=1)

# Effect:
momMatrix = getMomMatrix(priceAll)
momMatrix.head()

100%|██████████| 51/51 [00:02<00:00, 22.38it/s]


,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-05-17,-0.354887,-0.571186,-0.538372,-0.668143,-0.632540,-0.700956,-0.610182,-0.614177,-0.600658,...,-0.413020,NaN,-0.400037,-0.660054,-0.694803,-0.342545,-0.382088,-0.587151,-0.026398,-0.499388
2,2021-05-24,-0.382668,-0.286097,-0.058396,-0.460589,-0.400811,-0.550656,-0.489819,-0.473380,-0.382628,...,-0.343092,NaN,-0.231377,-0.510016,-0.456527,-0.555283,-0.363098,-0.403763,-0.013801,-0.268225
3,2021-05-31,0.232395,0.324998,0.577449,0.652091,0.058700,0.455930,0.128310,0.183153,0.347782,...,0.243890,NaN,0.024712,0.174587,0.321486,-0.207288,0.307871,0.311204,0.006451,0.307454
4,2021-06-07,-0.024113,-0.079155,0.106285,0.062860,-0.246951,0.113917,-0.135518,-0.058087,-0.081645,...,0.827227,NaN,0.088765,-0.083618,-0.012386,0.051829,-0.080879,-0.047386,0.001049,-0.136481


### Calculate MOM Percentile(30%, 70%)

In [141]:
# Only Select Assets(BTC, ETH...) Columns
col = []
for colName in momMatrix.columns:
     if colName not in ['Date','MOM30p','MOM70p']:
         col.append(colName)
momMatrix[col].head()

,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,BIX,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-0.354887,-0.571186,-0.538372,-0.668143,-0.632540,-0.700956,-0.610182,-0.614177,-0.600658,-0.574095,...,-0.413020,NaN,-0.400037,-0.660054,-0.694803,-0.342545,-0.382088,-0.587151,-0.026398,-0.499388
2,-0.382668,-0.286097,-0.058396,-0.460589,-0.400811,-0.550656,-0.489819,-0.473380,-0.382628,-0.504048,...,-0.343092,NaN,-0.231377,-0.510016,-0.456527,-0.555283,-0.363098,-0.403763,-0.013801,-0.268225
3,0.232395,0.324998,0.577449,0.652091,0.058700,0.455930,0.128310,0.183153,0.347782,0.219793,...,0.243890,NaN,0.024712,0.174587,0.321486,-0.207288,0.307871,0.311204,0.006451,0.307454
4,-0.024113,-0.079155,0.106285,0.062860,-0.246951,0.113917,-0.135518,-0.058087,-0.081645,0.011821,...,0.827227,NaN,0.088765,-0.083618,-0.012386,0.051829,-0.080879,-0.047386,0.001049,-0.136481


In [142]:
# 30% pencentile example
momMatrix[col].quantile(.3, axis=1).head()

0   -0.382874
1   -0.606894
2   -0.411377
3    0.149268
4   -0.084378
Name: 0.3, dtype: float64

In [143]:
momMatrix['MOM30p']=momMatrix[col].quantile(.3, axis=1)
momMatrix['MOM70p']=momMatrix[col].quantile(.7, axis=1)
momMatrix.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM,MOM30p,MOM70p
0,2021-05-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.382874,-0.382874
1,2021-05-17,-0.354887,-0.571186,-0.538372,-0.668143,-0.632540,-0.700956,-0.610182,-0.614177,-0.600658,...,-0.400037,-0.660054,-0.694803,-0.342545,-0.382088,-0.587151,-0.026398,-0.499388,-0.606894,-0.460275
2,2021-05-24,-0.382668,-0.286097,-0.058396,-0.460589,-0.400811,-0.550656,-0.489819,-0.473380,-0.382628,...,-0.231377,-0.510016,-0.456527,-0.555283,-0.363098,-0.403763,-0.013801,-0.268225,-0.411377,-0.277419
3,2021-05-31,0.232395,0.324998,0.577449,0.652091,0.058700,0.455930,0.128310,0.183153,0.347782,...,0.024712,0.174587,0.321486,-0.207288,0.307871,0.311204,0.006451,0.307454,0.149268,0.346496
4,2021-06-07,-0.024113,-0.079155,0.106285,0.062860,-0.246951,0.113917,-0.135518,-0.058087,-0.081645,...,0.088765,-0.083618,-0.012386,0.051829,-0.080879,-0.047386,0.001049,-0.136481,-0.084378,0.028558


In [144]:
# Reshape for Later Use
momPt = momMatrix[['MOM30p','MOM70p']]
momMatrix = momMatrix.drop(['MOM30p','MOM70p'], axis=1)

In [145]:
momPt.head()

,MOM30p,MOM70p
0,-0.382874,-0.382874
1,-0.606894,-0.460275
2,-0.411377,-0.277419
3,0.149268,0.346496
4,-0.084378,0.028558


### Calculate Momentum portfolio returns

In [146]:
# Delete the First 2 Rows of returnAll to Match the momMatrix Date
returnAll = returnAll.drop(returnAll.head(2).index).reset_index().drop('index', axis=1)
returnAll.head()

,Date,AAVE,ABT,AMPL,ANT,ARMOR,BADGER,BAL,BAND,BAT,...,VERI,WAX,WBTC,WTC,WaBi,YAM,YFI,ZRX,sUSD,wNXM
0,2021-05-10,0.143665,-0.171471,-0.281857,-0.244447,-0.323333,-0.256346,-0.182583,-0.204940,-0.194722,...,0.062182,NaN,-0.204564,-0.164128,-0.319218,0.064864,0.234226,-0.195655,-0.007335,-0.150428
1,2021-05-17,-0.435925,-0.482439,-0.357192,-0.560776,-0.456955,-0.597872,-0.523110,-0.514725,-0.504094,...,-0.447383,NaN,-0.245743,-0.593304,-0.551696,-0.382593,-0.499353,-0.486727,-0.019204,-0.410748
2,2021-05-24,0.094413,0.379361,0.464830,0.228098,0.103387,0.117416,0.069808,0.085198,0.244939,...,0.188722,NaN,0.019047,0.204790,0.212288,-0.279703,0.272157,0.161636,0.005509,0.241871
3,2021-05-31,0.126079,-0.039411,0.076882,0.345243,-0.040499,0.302944,0.054685,0.090265,0.082609,...,0.046409,NaN,0.005559,-0.025069,0.090076,0.100535,0.028074,0.128756,0.000937,0.052810
4,2021-06-07,-0.133376,-0.041374,0.027304,-0.209913,-0.215166,-0.145077,-0.180341,-0.136070,-0.151720,...,0.746187,NaN,0.082746,-0.060055,-0.093995,-0.044256,-0.105978,-0.156050,0.000112,-0.179796


In [151]:
def LMH_MOM(momMatrix, returnAll, priceAll):
    from tqdm import tqdm
    # Initialize DataFrames for Low, Medium and High MOM
    lowMom = pd.DataFrame(columns = ['Date',
                                     'LowMomentum'])
    lowMom['Date'] = momMatrix['Date']

    medMom = pd.DataFrame(columns = ['Date', 
                                     'MedMomentum'])
    medMom['Date'] = momMatrix['Date']

    highMom = pd.DataFrame(columns = ['Date', 
                                      'HighMomentum'])
    highMom['Date'] = momMatrix['Date']

    # Iterate Through Whole DataFrame to Get Low, Medium and High MOM
    # Iterate Though Rows
    for i in tqdm(range(0, momMatrix.shape[0])):

        # Initialization df Used in Method
        lowMomReturnList = []
        medMomReturnList = []
        highMomReturnList = []

        lowMomPriceList = []
        medMomPriceList = []
        highMomPriceList = []

        # Iterate Through Columns
        for j in range(1, momMatrix.shape[1]):
            if momMatrix.iloc[i,j] < momPt.iloc[i,0]:
                lowMomReturnList.append(returnAll.iloc[i,j])
                lowMomPriceList.append(priceAll.iloc[i,j])
            elif (momPt.iloc[i,0] < momMatrix.iloc[i,j]) and (momMatrix.iloc[i,j] < momPt.iloc[i,1]):
                medMomReturnList.append(returnAll.iloc[i,j])
                medMomPriceList.append(priceAll.iloc[i,j])
            elif momMatrix.iloc[i,j] > momPt.iloc[i,1]:
                highMomReturnList.append(returnAll.iloc[i,j])
                highMomPriceList.append(priceAll.iloc[i,j])

        # Low MOM
        lowValueWeightedAvgList = []
        lowMomPriceSum = sum(lowMomPriceList)
        for index in range(0, len(lowMomPriceList)):
            lowValueWeightedAvgList.append((lowMomPriceList[index]/lowMomPriceSum) * lowMomReturnList[index])
        lowMom.iloc[i,1] = sum(lowValueWeightedAvgList)
        
        # Med MOM
        medValueWeightedAvgList = []
        medMomPriceSum = sum(medMomPriceList)
        for index in range(0, len(medMomPriceList)):
            medValueWeightedAvgList.append((medMomPriceList[index]/medMomPriceSum) * medMomReturnList[index])
        medMom.iloc[i,1] = sum(medValueWeightedAvgList)

        # High MOM
        highValueWeightedAvgList = []
        highMomPriceSum = sum(highMomPriceList)
        for index in range(0, len(highMomPriceList)):
            highValueWeightedAvgList.append((highMomPriceList[index]/highMomPriceSum) * highMomReturnList[index])
        highMom.iloc[i,1] = sum(highValueWeightedAvgList)

        # Merge
        from functools import reduce
        momLMH = reduce(lambda  left,right:
                        pd.merge(left,right, 
                                on='Date',
                                how='outer'),
                        [lowMom, medMom, highMom])

    return momLMH

momLMH = LMH_MOM(momMatrix, returnAll, priceAll)

100%|██████████| 51/51 [00:01<00:00, 26.49it/s]


In [152]:
# from functools import reduce
# momLMH = reduce(lambda  left,right:
#                 pd.merge(left,right, 
#                          on='Date',
#                          how='outer'),
#                 [lowMom, medMom, highMom])
momLMH.head()

,Date,LowMomentum,MedMomentum,HighMomentum
0,2021-05-10,0,0,0
1,2021-05-17,-0.58298,-0.410799,-0.326323
2,2021-05-24,0.07534,0.262537,0.030213
3,2021-05-31,0.004954,0.040897,0.278781
4,2021-06-07,-0.172042,-0.107539,0.079781


### Calculate Momentum Factor

In [153]:
momFactor = pd.DataFrame(columns = ['Date', 'MOM_LTW'])
momFactor['Date'] = momMatrix['Date']
momFactor['MOM_LTW'] = momLMH['HighMomentum'] - momLMH['LowMomentum']

momFactor.head(20)

,Date,MOM_LTW
0,2021-05-10,0
1,2021-05-17,0.256657
2,2021-05-24,-0.045127
3,2021-05-31,0.273827
4,2021-06-07,0.251823
5,2021-06-14,0.052093
6,2021-06-21,0.189945
7,2021-06-28,0.177345
8,2021-07-05,0.006033
9,2021-07-12,0.05247


In [154]:
momLMH.HighMomentum.mean()

0.0636617749515979